In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip your_path_to_the_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "nmdr/llama-2-7b-chat-platypus-1k-Baymax10"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pub

tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

403

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1584

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  LLaMA-2 was trained on approximately 2 trillion tokens.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  LLaMA-2 was trained on approximately 2 trillion tokens.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the_zip_file
!unzip -q Questionscsv.zip -d questions

###General QA

In [ ]:
# General Questions

questions_file_path = '/content/questions/Questions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Newton's First Law of Motion, also known as the Law of Inertia, states that an object at rest
remains at rest, and an object in motion continues to move with a constant velocity unless acted upon by external forces.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The force of gravity between two objects changes with
distance according to Newton's Law of Universal Gravitation. The force of gravity between two objects decreases as the distance between them
increases. This relationship can be expressed mathematically through Newton's inverse square law formula: F = G \* (m_1 \* m_2) / r^2, where F is the
force of gravity, G is the gravitational constant, m_1 and m_2 are the masses of the two objects, and r is the distance between them.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Activation functions serve several purposes in neural networks:

1. Nonlinearity: They introduce nonlinearity into the model, allowing it to learn complex relationships between inputs and outputs. Without activation
functions, neural networks would only be able to learn linear relationships, limiting their ability to model complex systems.
2. Spatial compression: Some activation functions, like sigmoid and tanh, compress spatial information in the weight space, making it easier for the
network to learn hierarchical representations of data.
3. Steering behavior: Activation functions determine how the weights are updated during training. For example, ReLU steers the weights towards
positive values, while softmax steers them towards probabilities close to 1.
4. Energy function: The activation function defines the energy function of the network, which determines its overall performance. A good activation
function should minimize the energy function while maximizing expressiveness.

In su

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Supervised learning involves training a
machine learning algorithm on labeled data, where the target output is already known. The goal of supervised learning is to learn a mapping between
input features and corresponding outputs so that new inputs can be predicted accurately. On the other hand, unsupervised learning involves training an
algorithm on unlabeled data without any target output. The goal of unsupervised learning is to discover hidden patterns or relationships within the
data itself.

In summary, supervised learning focuses on predicting specific outputs given specific inputs, while unsupervised learning seeks to identify underlying
structures or patterns in the data without predetermined outputs.


Sources:
/content/papers/Attention.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundati

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Overfitting refers to when a neural network
learns the noise present within the training dataset rather than generalizing well to new unseen data. This occurs due to the model having too many
parameters relative to the amount of training data available, causing it to memorize specific instances rather than learning underlying patterns. As a
result, the model performs poorly on fresh test data. To avoid this issue, techniques like regularization, early stopping, dropout, and batch
normalization are employed during training to reduce overfitting.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  The learning rate plays a crucial role in training neural
networks. It determines how quickly the weights adapt to new data during 

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is Newton's first law of motion?
 response:   Newton's First Law of Motion, also known as the Law of Inertia, states that an object at rest
remains at rest, and an object in motion continues to move with a constant velocity unless acted upon by external forces.


Q1: How does the force of gravity between two objects change with distance?
 response:   The force of gravity between two objects changes with
distance according to Newton's Law of Universal Gravitation. The force of gravity between two objects decreases as the distance between them
increases. This relationship can be expressed mathematically through Newton's inverse square law formula: F = G \* (m_1 \* m_2) / r^2, where F is the
force of gravity, G is the gravitational constant, m_1 and m_2 are the masses of the two objects, and r is the distance between them.


Q2: What is the principle of conservation of energy?
 response:   The principle of conservation of energy states that energy cannot be created or
destroyed, 

In [ ]:
json_file_name = 'generalQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'generalQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Cosmology QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroCosmoQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Studying astrophysics and cosmology aim to understand the origins,
evolution, structure, and fate of the universe. By exploring the fundamental laws of physics that govern cosmic phenomena, astronomers and physicists
hope to gain insights into how matter, energy, space, time, and gravity interact within our cosmos. This knowledge can also shed light on some of
humanity's most profound questions, including the nature of existence itself. Ultimately, understanding more about the universe we inhabit can lead to
advancements in technology, medicine, engineering, and other fields that benefit society as a whole.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Augmented Reality in astrophysics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Big Bang theory is supported by several lines of evidence, including:

1. Cosmic Microwave Background Radiation: This is thought to be residual heat from the initial explosion, detected throughout the universe at a
temperature of about 2.7 degrees Kelvin.
2. Abundance of Light Elements: According to the Big Bang theory, these elements were formed during the first few minutes after the universe began,
when temperatures were high enough for nuclear reactions to occur. The abundance of light elements observed in the universe matches what would be
expected if they had been created in this way.
3. Large Scale Structure of the Universe: Observations of galaxy distributions and distribution of matter within them support the idea that the
universe started off very uniform and then evolved into its current state through gravitational collapse.
4. Redshift of Light from Distant Galaxies: The farther away a galaxy is, the faster its light is shifted towards the red end of the spectrum due t

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Recent developments in cosmology include the discovery of dark matter and dark
energy, which together account for approximately 95% of the universe's mass-energy budget. Additionally, observations of cosmic microwave background
radiation have revealed evidence for a flat global topology and support for the inflationary theory proposed by Alan Guth in the early 1980s. Other
notable advancements include the construction of large-scale surveys such as the Sloan Digital Sky Survey (SDSS) and the 2-degree Field (2dF) Catalog,
which will revolutionize the field by providing unprecedentedly detailed data on galaxy distributions and redshift patterns. Multi-object fiber
spectroscopy has also become increasingly popular due to its ability to simultaneously observe multiple objects within a single exposure.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/conten

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The observational revolution in cosmology refers to the significant
advancements made in measuring various cosmic phenomena over the past few decades, providing us with more accurate determinations of the parameters of
our standard cosmological model. This has been achieved through observations of redshift supernovae, gravitational lensing, large scale structure
measurements, distribution of galaxies, and many other factors. These new data constrain the parameter space of cosmological models, leading to a
greater understanding of the origins of the universe and its evolution.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Cosmological observations are explained by various theoretical models,
including the Big Bang theory, inflationary theories, dark matter models, and modified gravity theories. The standard Big Bang theory has been
supported by numerous lines of evidence, including the cosmic microwave background radiation, abundance of light elements, large-scale structure
formation, and redshift measurements of galaxy distances. However, it remains incomplete, as its predictions do not account for certain phenomena,
such as the accelerating expansion rate observed since the late 1990s. Inflationary theories attempt to address these discrepancies by introducing a
period of rapid expansion early in the universe's history, which could resolve some of these issues. Dark matter models propose the existence of non-
luminous matter that makes up approximately 85% of the universe's mass-energy budget but interacts only through gravitational forces. Modified gravity
theories seek to modify Einstein's general

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the goal of studying astrophysics and cosmology?
 response:   Studying astrophysics and cosmology aim to understand the origins,
evolution, structure, and fate of the universe. By exploring the fundamental laws of physics that govern cosmic phenomena, astronomers and physicists
hope to gain insights into how matter, energy, space, time, and gravity interact within our cosmos. This knowledge can also shed light on some of
humanity's most profound questions, including the nature of existence itself. Ultimately, understanding more about the universe we inhabit can lead to
advancements in technology, medicine, engineering, and other fields that benefit society as a whole.


Q1: What supports the Big Bang theory?
 response:   The Big Bang theory is supported by several lines of evidence, including:

1. Cosmic Microwave Background Radiation: This is thought to be residual heat from the initial explosion, detected throughout the universe at a
temperature of about 2.7 degrees Kelvi

In [ ]:
json_file_name = 'AstroCosmoQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroCosmoQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Physics QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroPhysicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Chandrasekhar limit refers to the maximum mass of a white
dwarf star beyond which it cannot sustain its own gravity due to degeneracy pressure from electrons. It was first proposed by Subrahmanyan
Chandrasekhar in 1930 and later confirmed through observations. The limit is around 1.4 solar masses (M☉) and represents the highest mass at which a
white dwarf can maintain stability without undergoing catastrophic collapse. Exceeding this limit results in either a supernova explosion or the
formation of a black hole. Understanding the Chandrasekhar limit plays crucial roles in understanding various phenomena such as binary star systems,
supernovae events, and the final stages of massive star evolution.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-phy

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  White dwarf stars play a crucial role in
understanding the final stages of stellar evolution. They are formed when a star with a mass similar to that of the sun reaches the end of its life
cycle and collapses into a dense ball of degenerate matter. This process can occur through either a violent implosion or a gradual change in the
stellar structure.

One key aspect of white dwarf stars is their extreme compactness, which arises from the immense pressure generated by their self-gravity. Unlike
regular stars, they cannot maintain equilibrium through pressure alone; therefore, they rely solely on their internal gravity to hold themselves
together. This makes them ideal candidates for studying the behavior of matter under intense gravitational conditions.

Another important contribution made by white dwarfs is providing insight into the concept of "degenerate matter." These objects represent the boundary
beyond which ordinary matter cannot exist due to its high density. By studying how 

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


Experimental advances that have allowed for the direct observation of gravitational waves include the Laser Interferometer Gravitational-Wave
Observatory (LIGO) and Virgo detectors, which use laser interferometry to measure tiny changes in distance caused by passing gravitational waves.
These observations were announced in 2015 and confirmed in 2017, marking the first detection of gravitational waves from merging black holes and
neutron stars.

These discoveries represent significant milestones in astrophysics because they provide new insights into the behavior of matter under extreme
conditions, such as those encountered near singularities in collapsing stars. They also offer opportunities to study cosmic phenomena that were
previously invisible to us, including the merger of binary systems containing massive objects like black holes or neutron stars. Additionally, these
findings could lead to further breakthroughs in understanding how gravity works at very small distances, potential

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The discovery of black holes
challenges our understanding of physics under extreme conditions by highlighting the limitations of current theories and raising questions about their
validity in such environments. For example, the existence of black holes suggests that gravity can become so strong near massive objects that it
creates a "singularity," where space-time becomes distorted beyond comprehension. This raises concerns about how well established theories like
general relativity hold up against such intense gravitational forces. Additionally, studying black holes provides insights into phenomena such as
Hawking radiation, which could potentially shed light on quantum mechanics and its role in high-energy processes. Overall, exploring the properties
and behavior of black holes pushes us closer towards developing a complete theory capable of explaining all aspects of gravity under extreme
conditions.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/paper

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Contemporary
challenges in theoretical physics suggested by the study of cosmological phenomena include understanding the origins of the universe, developing new
theories to explain dark matter and dark energy, resolving cosmic inflation, addressing the hierarchy problem, and reconciling quantum mechanics and
gravity.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
  There are several cutting-edge
technologies that are expected to revolutionize our understanding of cosmic phenomena in the coming years. Some of these include:

1. Next-generation telescopes: The Square Kilometre Array (SKA), scheduled to launch in the mid-2020s, promises unprecedented sensitivity and
resolution in radio astronomy. Similarly, the Giant Magellan Telescope (GMT) and Thirty Meter Te

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the Chandrasekhar limit and its significance in astrophysics?
 response:   The Chandrasekhar limit refers to the maximum mass of a white
dwarf star beyond which it cannot sustain its own gravity due to degeneracy pressure from electrons. It was first proposed by Subrahmanyan
Chandrasekhar in 1930 and later confirmed through observations. The limit is around 1.4 solar masses (M☉) and represents the highest mass at which a
white dwarf can maintain stability without undergoing catastrophic collapse. Exceeding this limit results in either a supernova explosion or the
formation of a black hole. Understanding the Chandrasekhar limit plays crucial roles in understanding various phenomena such as binary star systems,
supernovae events, and the final stages of massive star evolution.


Q1: How do white dwarf stars contribute to our understanding of stellar evolution?
 response:   White dwarf stars play a crucial role in
understanding the final stages of stellar evolution. They are f

In [ ]:
json_file_name = 'AstroPhysicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroPhysicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

### Attention Questions

In [ ]:
# Attention Questions

questions_file_path = '/content/questions/AttQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model introduces several innovative features compared to
traditional recurrent neural network (RNN) architectures:

1. Self-Attention Mechanism: Instead of relying solely on the previous hidden state, the Transformer uses self-attention mechanisms to compute
weighted combinations of all input tokens. This allows the model to capture longer-range dependencies more effectively.
2. Multi-Head Attention: The Transformer employs multiple attention heads simultaneously, each computing its own attention weights. By combining these
outputs, the model can focus on different aspects of the input sequence simultaneously.
3. Parallelization: Unlike RNNs, where processing one token requires scanning the entire sequence, the Transformer processes each token independently
thanks to parallelization across all positions within a head.
4. Encoder-Decoder Structure: The Transformer divides the input into two parts: an encoder that generates contextualized representations and a decoder
t

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model enhances training efficiency through several
innovative techniques:

1. Parallelization: Unlike traditional recurrent neural networks (RNNs), which process sequences sequentially, the Transformer processes them in
parallel across multiple GPUs or CPU cores. This allows for faster training times and larger batch sizes, leading to improved convergence rates.
2. Attention mechanism: Instead of relying solely on the last hidden layer of an RNN to compute the next word's probability distribution, the
Transformer uses self-attention mechanisms to consider all words in the sequence simultaneously. This reduces computation complexity and enables
faster training.
3. Multi-head attention: By introducing multiple attention heads, each head computes its own weighted sum of the input words, allowing the model to
focus on different aspects of the input sequence simultaneously. This further accelerates training speed without sacrificing performance.
4. Positional encoding: To 

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Attention in Transformers refers to a mechanism that enables the model to focus on specific parts
of the input when generating output. It is implemented by computing weights for each element in the input sequence relative to others, allowing the
model to selectively attend to relevant elements during processing. This helps improve overall performance by reducing the impact of irrelevant
information and increasing the importance of important details.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer uses several regularization techniques to prevent
overfitting and improve its performance. These include:

* Dropout: Applied to the output of each sub-layer before adding it to the sub-layer input and normalizing.
* Residual dropout: Applied to the sum of the embeddings and the positional encoding in both the encoder and decoder stacks.
* Multi-head attention: Used to attend to multiple positions simultaneously, which helps capture longer-range dependencies.
* Scaled dot-product attention: Implemented to reduce computational complexity without sacrificing too much accuracy.
* Parameter-free position representation: Introduced to eliminate the need for explicit positional encoding.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model improved machine translation accuracy by
replacing traditional recurrent neural networks (RNNs) with self-attention mechanisms. This allowed the model to focus more effectively on specific
parts of the input sequence, leading to better overall performance. Additionally, the Transformer model introduced multi-head attention, which further
increased its ability to capture complex relationships between different parts of the input sequence. Finally, the Transformer model was able to train
faster than previous architectures due to its parallelization capabilities, allowing it to reach higher levels of accuracy in shorter periods of time.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
  Yes, the Transformer model can be applied beyond machine
translation. In fact, its success in natural language processing (NLP) has led to its adoption in various areas su

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What innovation does the Transformer model introduce?
 response:   The Transformer model introduces several innovative features compared to
traditional recurrent neural network (RNN) architectures:

1. Self-Attention Mechanism: Instead of relying solely on the previous hidden state, the Transformer uses self-attention mechanisms to compute
weighted combinations of all input tokens. This allows the model to capture longer-range dependencies more effectively.
2. Multi-Head Attention: The Transformer employs multiple attention heads simultaneously, each computing its own attention weights. By combining these
outputs, the model can focus on different aspects of the input sequence simultaneously.
3. Parallelization: Unlike RNNs, where processing one token requires scanning the entire sequence, the Transformer processes each token independently
thanks to parallelization across all positions within a head.
4. Encoder-Decoder Structure: The Transformer divides the input into two parts: an 

In [ ]:
json_file_name = 'AttQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AttQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Coherent Spin Questions

In [ ]:
# Coherent Spin Questions

questions_file_path = '/content/questions/CoSpinQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The objective of using Heisenberg exchange in quantum
computing is to transmit the spin state of an electron back and forth along an array of electrons in a quadruple quantum dot. This allows for the
transfer of single-spin eigenstates and entangled states via coherent SW AP gates between all neighboring pairs of spins in a four-qubit array.
Additionally, this method is scalable to large numbers of qubits, making it suitable for multi-qubit gates and quantum error correction in large spin-
qubit arrays.


Sources:
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  External factors influencing
the Heisenberg exchange interaction during quantum spin-state transfer include magnetic fields, temperature, and nuclear spin noise.


Sources:
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Quantum scalability refers to the ability of a
quantum system to handle increasing numbers of qubits while maintaining its functionality and performance. In the context of coherent spin-state
transfer via Heisenberg exchange, scaling up the number of qubits presents several challenges. One major issue is ensuring enough connectivity among
the qubits so they can efficiently communicate with each other and perform quantum computations. Another concern is managing the increased complexity
of controlling multiple qubits simultaneously.

To address these issues, researchers have proposed various strategies, including using larger quantum dots or arrays of smaller dots, optimizing the
design of the quantum circuit, and developing new techniques for measuring and controlling the interactions between qubits. By improving the
scalability of quantum systems, scientists hope to advance their development towards practical applications in fields like cryptography, drug
discovery, and optimization

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Heisenberg exchange plays a crucial role in
quantum computing applications, particularly in spin-based quantum computing. It allows for the transfer of spin states between neighboring electrons
in an array of spin qubits, enabling multi-qubit gates and quantum error correction in large spin-qubit arrays. The Heisenberg exchange interaction
tends to swap the spin states of neighboring electrons, creating a means for coherently transferring spin states along an array of electrons in a
quadruple quantum dot. This property enables the implementation of exchange-based spin-state transfer, which can be useful in scaling up spin-based
quantum processors for more robust and reliable quantum computing capabilities.


Sources:
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  In the article "Quantum Computation and Quantum
Information," authors Nielsen and Chuang use several metaphors to explain the initialization of quantum systems. They compare the process of
initializing a quantum computer to a game of hide-and-seek, where the particles representing the qubits must find their hiding places within the
system. Additionally, they liken the initialization process to a "quantum version of a classical algorithm" [1], suggesting that while the underlying
principles may differ, the overall approach is similar to those used in classical computing.

Another metaphor used in the article is that of a "quantum jumping jack." Here, the authors describe how a quantum particle can exist in multiple
locations simultaneously, much like a person performing acrobatic jumps across different points on a trampoline. This analogy helps illustrate the
concept of superposition, which allows quantum particles to occupy multiple states simultaneously until observed [1].

Finally,

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the objective of using Heisenberg exchange in quantum computing?
 response:   The objective of using Heisenberg exchange in quantum
computing is to transmit the spin state of an electron back and forth along an array of electrons in a quadruple quantum dot. This allows for the
transfer of single-spin eigenstates and entangled states via coherent SW AP gates between all neighboring pairs of spins in a four-qubit array.
Additionally, this method is scalable to large numbers of qubits, making it suitable for multi-qubit gates and quantum error correction in large spin-
qubit arrays.


Q1: What external factors influence the Heisenberg exchange interaction during quantum spin-state transfer?
 response:   External factors influencing
the Heisenberg exchange interaction during quantum spin-state transfer include magnetic fields, temperature, and nuclear spin noise.


Q2: How does coherent spin-state transfer contribute to advancements in quantum technology?
 response:   Coherent 

In [ ]:
json_file_name = 'CoSpinQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'CoSpinQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)